In [1]:
import sys
import os
import cv2
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input, decode_predictions
# from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

import tensorflow.keras as keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import optimizers
from keras.datasets import cifar100
import tensorflow as tf
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import time
from skimage.transform import resize
# from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator

In [2]:
num_classes = 29
nb_epochs = 10
img_sz = (100, 100)

# data paths
train_path = '../../data/asl_alphabet_train/'
validation_path = '../../data/asl_alphabet_validation/'

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.3)
valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=img_sz,
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical',
        subset='training')
test_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=img_sz,
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical',
        subset='validation')
validation_generator = valid_datagen.flow_from_directory(
        validation_path,
        target_size=img_sz,
        color_mode='rgb',
        batch_size=32,
        class_mode='categorical')

Found 60901 images belonging to 29 classes.
Found 26100 images belonging to 29 classes.
Found 30 images belonging to 29 classes.


In [3]:
next(train_generator)[0].shape

(32, 100, 100, 3)

In [4]:
base_model = MobileNetV2(input_shape=(48,48,3),weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dense(1024,activation='relu')(x) #dense layer 2
x = Dense(512,activation='relu')(x) #dense layer 3
preds = Dense(num_classes,activation='softmax')(x) #final layer with softmax activation

model = Model(inputs=base_model.input,outputs=preds)

model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 49, 49, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 24, 24, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 24, 24, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

In [5]:
# for layer in model.layers:
#     layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [6]:
model.compile(optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
early_stop = keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                           min_delta=0,
                                           patience=2,
                                           verbose=0,
                                           mode="auto",
                                           baseline=None,
                                           restore_best_weights=False)

callbacks = [early_stop]
t=time.time()
historytemp = model.fit(train_generator,
                        steps_per_epoch=len(train_generator),
                        epochs=10,
                        validation_data=test_generator,
                        callbacks=callbacks)
print('Training time: %s' % (t - time.time()))

Epoch 1/10
1904/1904 [==============================] - 1231s 647ms/step - loss: 0.5729 - accuracy: 0.8258 - val_loss: 5.2875 - val_accuracy: 0.0353
Epoch 2/10
1904/1904 [==============================] - 1098s 577ms/step - loss: 0.0960 - accuracy: 0.9693 - val_loss: 3.6740 - val_accuracy: 0.3084
Epoch 3/10
1904/1904 [==============================] - 1117s 586ms/step - loss: 0.0667 - accuracy: 0.9796 - val_loss: 1.8707 - val_accuracy: 0.6162
Epoch 4/10
1904/1904 [==============================] - 1100s 578ms/step - loss: 0.0516 - accuracy: 0.9841 - val_loss: 2.2480 - val_accuracy: 0.5687
Epoch 5/10
1904/1904 [==============================] - 1110s 583ms/step - loss: 0.0393 - accuracy: 0.9886 - val_loss: 1.0025 - val_accuracy: 0.8007
Epoch 6/10
1904/1904 [==============================] - 1135s 596ms/step - loss: 0.0368 - accuracy: 0.9891 - val_loss: 0.8366 - val_accuracy: 0.8199
Epoch 7/10
1904/1904 [==============================] - 1116s 586ms/step - loss: 0.0317 - accuracy: 0.9911

In [8]:
model.save('models/mobilenet.h5')

model_json = model.to_json()
with open("mobilenet.json", "w") as json_file:
    json_file.write(model_json)
